In [25]:
import tensorflow as tf

In [28]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5009171127313588672
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 22729785344
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 657482955967892190
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [26]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

Physical devices cannot be modified after being initialized


In [2]:
from tensorflow.keras.datasets import imdb
(train_input, train_target), (test_input, test_target) = imdb.load_data(num_words=1000)

In [3]:
print(train_input.shape, test_input.shape)

(25000,) (25000,)


In [4]:
import numpy as np

In [5]:
lengths = np.array([len(x) for x in train_input])

In [6]:
lengths

array([218, 189, 141, ..., 184, 150, 153])

In [7]:
from sklearn.model_selection import train_test_split
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.2)

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
train_seq = pad_sequences(train_input, maxlen=500)
val_seq = pad_sequences(val_input, maxlen=500)

In [9]:
print(train_seq.shape)

(20000, 500)


In [10]:
train_seq

array([[  0,   0,   0, ...,  30, 110, 133],
       [  0,   0,   0, ..., 701, 570,   2],
       [  0,   0,   0, ..., 283, 406,   2],
       ...,
       [  0,   0,   0, ...,  49, 128,  20],
       [ 27, 226,   2, ..., 339,  12,  49],
       [  0,   0,   0, ..., 479,   7, 363]])

# 모델 만들기

In [20]:
from tensorflow.keras import Sequential, layers, utils, optimizers, callbacks

In [12]:
train_oh = utils.to_categorical(train_seq)

In [13]:
val_oh = utils.to_categorical(val_seq)

In [14]:
model = Sequential()
model.add(layers.SimpleRNN(8, input_shape=(2000, 500)))
model.add(layers.Dense(1, activation='sigmoid'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 8)                 4072      
                                                                 
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 4,081
Trainable params: 4,081
Non-trainable params: 0
_________________________________________________________________


In [18]:
rmsprop = optimizers.RMSprop(learning_rate = 1e-4)

In [21]:
model.compile(optimizer=rmsprop, loss='binary_crossentropy', metrics=['accuracy'])
checkpoint_cb = callbacks.ModelCheckpoint('best-simplernn-model.h5')
early_stopping_cb = callbacks.EarlyStopping(patience=3, restore_best_weights=True)


In [24]:
history = model.fit(train_oh, train_target, epochs=100, batch_size=64, validation_data=(val_oh, val_target), callbacks=[checkpoint_cb, early_stopping_cb])

ResourceExhaustedError: {{function_node __wrapped____EagerConst_device_/job:localhost/replica:0/task:0/device:GPU:0}} SameWorkerRecvDone unable to allocate output tensor. Key: /job:localhost/replica:0/task:0/device:CPU:0;e7a15e0b6f82c50c;/job:localhost/replica:0/task:0/device:GPU:0;edge_2_input;0:0